
# Data and computations for paper 

Lars G. Johnsen
National Library of Norway

In [1]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
mu.update('collocations')
from collocations import dist_coll_urn, urn_coll, calculate_midpoint, dist
import pandas as pd

Updated file `C:\Users\yoons\Documents\GitHub\newspapers_coll_conc\collocations.py`

In [2]:
def df_jaccard(df, col1, col2, number=100, asc=False):
    s1 = df.sort_values(by=col1, ascending=asc)[:number].index
    s2 = df.sort_values(by=col2, ascending=asc)[:number].index
    
    #print(len(set(s1)&set(s2)))
    
    return jaccard(s1, s2)

In [3]:
def jaccards(s1, s2):
    return len(set(s1)&set(s2)),len(set(s1)|set(s2))

In [4]:
def jaccard(s1, s2):
    return len(set(s1)&set(s2))/len(set(s1)|set(s2))

In [5]:
def make_coll_df(small, large, tot):
    """take two collocations, small and large, and a reference tot. The reference must av a column call tot as well"""
    coll = pd.DataFrame()
    coll['small'] = small['freq']
    coll['large'] = large['freq']
    #coll['ratio'] = coll['small']/coll['large']
    coll['srel'] = coll.small/coll.small.sum()
    coll['lrel'] = coll.large/coll.large.sum()
    coll['ratio'] = coll.srel/coll.lrel
    coll['adjusted'] = coll.small**0.01*coll.ratio
    coll['nb'] = coll.small/tot.tot
    return coll

In [6]:
import pandas as pd

In [7]:
def make_dcoll_df(dists, norm, tot):
    
    return nb.frame(
        {
        'score':(dists.freq/dists.freq.sum())**1*abs(norm/dists['2']), 
        'dist':dists['1'],
        'nb':(dists.freq/dists.freq.sum())/tot.tot
        }
    ).transpose()

### Reference corpus

The reference is the total counts from approximately 450 000 books from nb.no

In [8]:
tot = nb.frame(nb.totals(50000), 'tot')

In [9]:
nb.normalize_corpus_dataframe(tot)
tot.head()

,tot
.,0.058921
",",0.051453
og,0.025297
i,0.021425
det,0.012728


# Corpus

The corpus for doing collocations is a sample of 800 books from fictional literature, dewey decimal code 813.

In [10]:
smd = 5
lmd = 10
collword = 'motor'

In [11]:
def filenames(smd = smd, lmd = lmd, collword = collword, corpus_size= 1000, period = (1980, 2000)):
    return {
    'small_right': '_'.join([collword, str(smd), str(corpus_size), '-'.join([str(x) for x in period]), str(0), str(int(2*smd)), '.csv']),
    'large_right': '_'.join([collword, str(smd), str(corpus_size),  '-'.join([str(x) for x in period]), str(0), str(int(2*lmd)), '.csv']),
    'small_left' : '_'.join([collword, str(lmd), str(corpus_size),  '-'.join([str(x) for x in period]), str(int(2*smd)), str(0), '.csv']),
    'large_left' : '_'.join([collword, str(lmd), str(corpus_size),  '-'.join([str(x) for x in period]), str(int(2*lmd)), str(0), '.csv'])
        }

# Store the data subsequent use

In [12]:
names = filenames()
names

{'small_right': 'motor_5_1000_1980-2000_0_10_.csv',
 'large_right': 'motor_5_1000_1980-2000_0_20_.csv',
 'small_left': 'motor_10_1000_1980-2000_10_0_.csv',
 'large_left': 'motor_10_1000_1980-2000_20_0_.csv'}

In [13]:
a1 = pd.read_csv(names['small_right'], index_col=0)
a2 = pd.read_csv(names['large_right'], index_col=0)
b1 = pd.read_csv(names['small_left'], index_col=0)
b2 = pd.read_csv(names['large_left'], index_col=0)


## Create collocation dataframe 

Based on data from a1 and a2, and b1 and b2. The name for the collocations after is `coll` while `collb` is for the collocates coming before.

In [14]:
coll = make_dcoll_df(a1, lmd, tot)

In [15]:
collb = make_dcoll_df(b1, lmd, tot)

## Sorting 

Sorting on the reference is by column 'nb'

In [16]:
coll.sort_values(by='score', ascending=False)[:20][:20].fillna(0).style.background_gradient()

,score,dist,nb
.,0.135761,4.854661,1.119799
",",0.111874,4.494434,0.978429
og,0.068294,4.522542,1.220228
i,0.044519,4.910095,1.020234
som,0.034366,4.546720,1.266837
en,0.024214,5.588512,1.255277
er,0.023999,5.643117,1.168823
på,0.022570,5.457968,1.285126
for,0.021962,4.820473,1.123235
med,0.020125,5.257849,1.246535


In [17]:
coll.sort_values(by='nb', ascending=False)[:20].fillna(0).style.background_gradient()

,score,dist,nb
traktor,0.004085,5.439542,576.283711
Br.nr,0.000920,6.666667,471.487779
generator,0.002333,4.286111,464.778594
turtall,0.000843,5.775000,425.446413
tomgang,0.001422,3.222222,369.456693
Motoren,0.001570,4.853968,366.417291
motor,0.006221,6.568114,333.655179
vehicles,0.002786,1.000000,328.531033
motorvogner,0.000680,5.333333,324.951755
motorens,0.000372,7.944444,245.906948


In [18]:
coll.loc[[',','.', 'og','eller', 'i', 'på']]

,score,dist,nb
",",0.111874,4.494434,0.978429
.,0.135761,4.854661,1.119799
og,0.068294,4.522542,1.220228
eller,0.011019,4.277450,1.721850
i,0.044519,4.910095,1.020234
på,0.022570,5.457968,1.285126


# collb

In [19]:
collb.sort_values(by='nb', ascending=False)[:20].fillna(0).style.background_gradient()

,score,dist,nb
primus,0.033732,-1.040404,3163.753192
Maskiner,0.002637,-7.234043,681.857162
generator,0.001870,-4.941964,427.596306
motor,0.006221,-6.568114,333.655179
skrog,0.001068,-4.027778,318.077276
Damp,0.001258,-2.562500,288.418078
Motor,0.000932,-6.569444,271.634961
virkningsgrad,0.000831,-5.333333,271.287117
elektrisk,0.045014,-1.460998,260.166774
hydraulisk,0.002918,-1.488095,249.106961


In [20]:
collb.sort_values(by='score', ascending=False)[:40].fillna(0).style.background_gradient()

,score,dist,nb
en,0.112233,-3.240533,3.372387
.,0.078920,-6.425324,0.859907
",",0.071739,-5.973911,0.832371
med,0.054797,-3.924020,2.535890
elektrisk,0.045014,-1.460998,260.166774
og,0.044222,-5.075628,0.888019
av,0.035487,-4.927562,1.598008
primus,0.033732,-1.040404,3163.753192
som,0.027573,-5.291532,1.181726
i,0.025969,-6.135067,0.743017


# coll

# jaccard similarity

compare over a range

In [21]:
rng = range(2,40,2)

# for collb

In [22]:
jaccard_scoresb = nb.frame({'ratio': {x:df_jaccard(collb, 'nb', 'score', x) for x in rng}
                            }).transpose()

Text(0, 0.5, 'Jaccard score')

In [23]:
ax = jaccard_scoresb['ratio'].plot(title='compared to reference corpus');
ax.set_xlabel("Number of words")
ax.set_ylabel("Jaccard score")

# for coll

In [24]:
jaccard_scores = nb.frame({'ratio': {x:df_jaccard(coll, 'nb', 'score', x) for x in rng}     }).transpose()

Text(0, 0.5, 'Jaccard score')

In [25]:
ax = jaccard_scores['ratio'].plot(title='compared to reference corpus');
ax.set_xlabel("Number of words")
ax.set_ylabel("Jaccard score")